In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.47.0
Uninstalling transformers-4.47.0:
  Successfully uninstalled transformers-4.47.0
Found existing installation: accelerate 1.2.0
Uninstalling accelerate-1.2.0:
  Successfully uninstalled accelerate-1.2.0
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.47.0-py3-none-any.whl (10.1 MB)
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)


In [4]:
!pip install numpy<2.0


The system cannot find the file specified.


In [6]:
!pip install evaluate


In [31]:
from transformers import pipeline, set_seed
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd
from evaluate import load  # Updated import

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mdhar_hxm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [57]:
# "google/pegasus-cnn_dailymail" hugging face model for text summarization.
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import requests  # library for making http request
import zipfile   # library for working with zip files
import os        # library for interacting with the operating system

# Define the URL and output file paths
url = "https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip" # url of the zip file to download
zip_path = "summarizer-data.zip" #local file path to save the downloaded zip file

# Download the file
response = requests.get(url)        # send a GET request to download the file
with open(zip_path, "wb") as file:  # open a file in write-binary mode
    file.write(response.content)    # write the downloaded file content to disk

# Unzip the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:  # open the zip file in read mode
    zip_ref.extractall(".")                      # extract all files to the current directory

# Check extracted contents
print("Files extracted:", os.listdir("."))  # printing a list of files in the current directory


Files extracted: ['.ipynb_checkpoints', 'samsum-test.csv', 'samsum-train.csv', 'samsum-validation.csv', 'samsum_dataset', 'summarizer-data.zip', 'Untitled.ipynb']


In [21]:
dataset_samsum = load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [22]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary")

print(dataset_samsum["test"][1]["summary"])

split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [23]:
# converting text into the features based on vocaburary which proives model which i load in this project.
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

  return {
      'input_ids' : input_encodings['input_ids'], # Responsible for text repersentation based on vocabury size
      'attention_mask': input_encodings['attention_mask'], # responsible for indentify real data after padding
      'labels': target_encodings['input_ids']  # take target values from the input_ids
  }

In [24]:
dataset_samsum_pt =  dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

C:\Users\mdhar_hxm\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [25]:
# 3 more columns are add after converting text into the features repersentation.
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [26]:
# Training 

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [27]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

C:\Users\mdhar_hxm\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

C:\Users\mdhar_hxm\AppData\Local\Temp\ipykernel_12680\3860022235.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


In [29]:
trainer.train()

Step,Training Loss,Validation Loss


C:\Users\mdhar_hxm\anaconda3\Lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=51, training_loss=3.022708163541906, metrics={'train_runtime': 3393.1036, 'train_samples_per_second': 0.241, 'train_steps_per_second': 0.015, 'total_flos': 313450454089728.0, 'train_loss': 3.022708163541906, 'epoch': 0.9963369963369964})

In [58]:
def generate_batch_size_chunk(list_of_elements, batch_size):
    """Split the dataset into smaller batches that we can process
    simultaneously. Yield successive batch-size chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i: i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=None,
                                column_text='article',
                                column_summary='highlights'):
    """
    Calculate metrics on the test dataset.
    """
    if device is None:
        raise ValueError("Device is not specified. Provide a device (e.g., 'cuda' or 'cpu').")

    article_batches = list(generate_batch_size_chunk(dataset[column_text], batch_size))
    target_batches = list(generate_batch_size_chunk(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding='max_length', return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                    attention_mask=inputs["attention_mask"].to(device),
                                    length_penalty=0.8, num_beams=8, max_length=128)
        ''' Parameters for length penalty ensure that the model does not generate overly long sequences. '''
        # Decode the generated text
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                              clean_up_tokenization_spaces=True)
                             for s in summaries]

        # Add predictions and references to the metric
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Compute and return the ROUGE score
    score = metric.compute()
    return score


In [59]:
rouge_name = ["rouge1","rouge2","rougL","rougeLsum"]
rouge_metric = evaluate.load("rouge")

In [60]:
#!pip install evaluate


In [61]:
import evaluate  # New library for metrics
from tqdm import tqdm


In [66]:
rouge_name = ["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric = evaluate.load("rouge")

In [67]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, 
    batch_size=2, device=device, column_text="dialogue", column_summary="summary"
)

# Extract ROUGE scores
rouge_dict = {rn: score[rn] for rn in rouge_name}
pd.DataFrame(rouge_dict, index=["pegasus"])


100%|██████████| 5/5 [03:22<00:00, 40.52s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.263935,0.057306,0.204846,0.206736


In [68]:
# save model
model_pegasus.save_pretrained("pegasus-samsum-model")

C:\Users\mdhar_hxm\anaconda3\Lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [69]:
## save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\spiece.model',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [71]:
## load
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [73]:
# prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length":128}

smaple_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)



##
print("Dialogue")
print(smaple_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(smaple_text, **gen_kwargs)[0]["summary_text"])


Device set to use cpu
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
